In [1]:
import misfunciones, itertools

### Lab 7 - Optimizando y acelerando nuestros algoritmos 🏎🏁

En la última parte del lab 6, vimos cómo efectuar una búsqueda _difusa_ de k-meros en una secuencia empleando la distancia de Hamming, que calculábamos en la función `distancia(kmero_1, kmero_2)`:

In [2]:
def distancia(kmero_1, kmero_2):
    contador_difs = 0
    for c, b in zip(kmero_1, kmero_2):
        if c != b:
            contador_difs += 1
    return contador_difs

Usando `distancia(kmero_1, kmero_2)`, pudimos implementar otra función `cuenta_kmero_aprox(secuencia, kmero, d)` que devolvía cuántas veces aparecía un k-mero en una secuencia permitiendo variaciones en `d` letras:

In [3]:
def cuenta_kmero_aprox(secuencia, kmero, d):
    cuenta = 0
    for i in range(len(secuencia) - len(kmero) + 1):
        if distancia(secuencia[i:i+len(kmero)], kmero) <= d:
            cuenta += 1
        if distancia(secuencia[i:i+len(kmero)], misfunciones.inv_comp(kmero)) <= d:
            cuenta += 1
    return cuenta

Finalmente, generamos un _mapa de frecuencias_ para k-meros permitiendo al menos variaciones en `d` letras con la función:

In [4]:
def kmeros_frecuentes_aprox(secuencia, k, d):
    freq = {}
    n = len(secuencia)
    todos_kmeros = list(["".join(kmero) for kmero in itertools.product("ACGT", repeat=k)])
    for kmero in todos_kmeros:
        frec = cuenta_kmero_aprox(secuencia, kmero, d)
        freq[kmero] = frec
    return freq

In [5]:
def lee_genoma(ruta_fichero):
  fd = open(ruta_fichero)
  genome = fd.read()
  fd.close()
  return genome

In [6]:
ec_genoma = lee_genoma('../data/E-coli.txt')

In [7]:
%time resultados  = kmeros_frecuentes_aprox(ec_genoma[3923620:3923620+500], 9, 1) # tarda mucho! ¿Cómo lo arreglamos?

KeyboardInterrupt: 

Sin embargo, también pudimos comprobar que este algoritmo tardaba bastante en ejecutarse, ya un requisito era generar previamente los $4^{k}$ k-meros posibles. Una pregunta que quedó en el aire era cómo podíamos acelerar este algoritmo: vamos a verlo. 

## Optimizando
La optimización en este caso la vamos a conseguir considerando sólo el grupo de k-meros que tengan una distancia `d` o inferior a cualquier k-mero que aparezca en la secuencia. A estos grupos, los denominaremos `d-vecindarios`.

Por ejemplo, el 1-vecindario (vecinos inmediatos) del 3-mero `ACG` es: `{ACG, ACG CCG GCG TCG AAG AGG ATG ACA ACC ACT}` (nótese que se incluye el "vecino" originario del vecindario). 

Como ves, el tamaño del d-vecindario de un k-mero dependerá del número de símbolos del alfabeto y la longitud del k-mero.

__Pregunta: ¿Cuántos vecinos tendrá el vecindario de un k-mero (para el alfabeto del ADN)?__


Vamos a implementar la función `vecinos_directos(kmero, alfabeto)`, que nos devolverá el conjunto de vecinos directos para un k-mero y un alfabeto:

In [17]:
def vecinos_directos(kmero, alfabeto):
    vecindario = set([kmero])
    elementos_kmero = list(kmero)
    for i, b in enumerate(elementos_kmero):
        for letra in alfabeto:
            if letra == b:
                continue
            vecino = elementos_kmero[:]
            vecino[i] = letra
            vecindario.add("".join(vecino))
    return vecindario

In [18]:
vecinos_directos('ACG', ('A', 'C', 'T', 'G'))

{'AAG', 'ACA', 'ACC', 'ACG', 'ACT', 'AGG', 'ATG', 'CCG', 'GCG', 'TCG'}

### d-vecindarios
La función que hemos implementado genera el 1_vecindario (vecinos directos) para un k-mero, ¡pero recuerda que la búsqueda difusa puede permitir más modificaciones!

__Pregunta: Piensa cómo generarías el d-vecindario de un k-mero dado. Acaba de implementar la función que te doy en la siguiente celda.__


In [19]:
def vecindario(kmero, alfabeto, d):
    vecindario = set([kmero])
    for i in range(d):
        i_vecinos = set()
        for vecino in vecindario:
            i_vecinos = i_vecinos | vecinos_directos(vecino, alfabeto)
        vecindario = vecindario | i_vecinos
    return vecindario 

In [20]:
vecindario('ACGT', ('A', 'C', 'T', 'G'), 2)

{'AAAT',
 'AACT',
 'AAGA',
 'AAGC',
 'AAGG',
 'AAGT',
 'AATT',
 'ACAA',
 'ACAC',
 'ACAG',
 'ACAT',
 'ACCA',
 'ACCC',
 'ACCG',
 'ACCT',
 'ACGA',
 'ACGC',
 'ACGG',
 'ACGT',
 'ACTA',
 'ACTC',
 'ACTG',
 'ACTT',
 'AGAT',
 'AGCT',
 'AGGA',
 'AGGC',
 'AGGG',
 'AGGT',
 'AGTT',
 'ATAT',
 'ATCT',
 'ATGA',
 'ATGC',
 'ATGG',
 'ATGT',
 'ATTT',
 'CAGT',
 'CCAT',
 'CCCT',
 'CCGA',
 'CCGC',
 'CCGG',
 'CCGT',
 'CCTT',
 'CGGT',
 'CTGT',
 'GAGT',
 'GCAT',
 'GCCT',
 'GCGA',
 'GCGC',
 'GCGG',
 'GCGT',
 'GCTT',
 'GGGT',
 'GTGT',
 'TAGT',
 'TCAT',
 'TCCT',
 'TCGA',
 'TCGC',
 'TCGG',
 'TCGT',
 'TCTT',
 'TGGT',
 'TTGT'}

### Sólo los vecinos
Ahora que ya sabemos encontrar el d-vecindario de un k-mero, podemos modificar nuestro algoritmo para generar mapas de frecuencias siguiendo nuestro razonamiento inicial. 

__Pregunta: Modifica la función kmeros_frecuentes_aprox para que considere sólo los k-meros que sean miembros de algún vecindario de todos los k-meros que aparezcan en la secuencia. Mide el tiempo que tarda en ejecutarse con la magia `%time`. ¿Es esta diferencia la que esperabas?__

In [22]:
def kmeros_frecuentes_aprox_vecindarios(secuencia, k, d, alfabeto):
    freq = {}
    cercanos = []
    
    for i in range(len(secuencia) - k + 1):
        vecindario_actual = vecindario(secuencia[i:i+k], alfabeto, d)
        for vecino in vecindario_actual:
            cercanos.append(vecino)
    
    cercanos = list(set(cercanos))
    for cercano in cercanos:
        freq[cercano] = cuenta_kmero_aprox(secuencia, cercano, d)
    
    return freq

In [28]:
%time resultados = kmeros_frecuentes_aprox_vecindarios(ec_genoma[3923620:3923620+500], 9, 1, ('A', 'C', 'T', 'G'))

CPU times: user 30.6 s, sys: 115 ms, total: 30.7 s
Wall time: 30.8 s


In [29]:
len(resultados)

13132

In [30]:
valor_max = max(resultados.values())
top_kmeros = []
for k,v in resultados.items():
    if v == valor_max:
        top_kmeros.append(k)

print(top_kmeros, valor_max)

['GTGGATAAC', 'GATCAACAG', 'TTATTCACA', 'AGATCTCTT', 'CCAGGATCC', 'CTGTTGATC', 'CAGAAGATC', 'AGCTGGGAT', 'CTGGGATCA', 'TGATCCCAG', 'GGATCCTGG', 'GCTGGGATC', 'TCTGGATAA', 'TGTGAATAA', 'AATGATCCG', 'GTTATCCAC', 'TGGATAACC', 'TGTGGATAA', 'AAGGATCCT', 'AAGAGATCT', 'AGGATCCTT', 'AGAACAACA', 'CGGATCATT', 'GATCTTCTG', 'TGATCAACA', 'TTATCCACA', 'AGGATCAAC', 'TGTTGTTCT', 'TTATCCAGA', 'GTTGATCCT', 'TGTTGATCA', 'GGTTATCCA', 'ATCCCAGCT', 'GATCCCAGC'] 4


In [31]:
def agrupa_resultados(top_kmeros):
    grupos = []
    while len(top_kmeros) > 0:
        kmero = top_kmeros.pop()
        grupo_temp = [kmero]
        for i in range(len(top_kmeros)):
            if distancia(kmero, top_kmeros[i]) <= 1 or kmero == misfunciones.inv_comp(top_kmeros[i]) or distancia(misfunciones.inv_comp(kmero), top_kmeros[i]) <= 1:
                grupo_temp.append(top_kmeros[i])

        top_kmeros = [kmero for kmero in top_kmeros if kmero not in grupo_temp]
        grupos.append(sorted(grupo_temp)) # ordenamos cada grupo lexicográficamente para ver sus diferencias.
    
    return sorted(grupos, key=lambda kmeros: kmeros[0]) # mantenemos el orden lexicográfico en la lista de grupos

In [32]:
agrupa_resultados(top_kmeros)

[['AAGAGATCT', 'AGATCTCTT'],
 ['AAGGATCCT', 'AGGATCCTT'],
 ['AATGATCCG', 'CGGATCATT'],
 ['AGAACAACA', 'TGTTGTTCT'],
 ['AGCTGGGAT', 'ATCCCAGCT'],
 ['AGGATCAAC', 'GTTGATCCT'],
 ['CAGAAGATC', 'GATCTTCTG'],
 ['CCAGGATCC', 'GGATCCTGG'],
 ['CTGGGATCA', 'TGATCCCAG'],
 ['CTGTTGATC', 'GATCAACAG'],
 ['GATCCCAGC', 'GCTGGGATC'],
 ['GGTTATCCA', 'TGGATAACC'],
 ['GTGGATAAC', 'GTTATCCAC'],
 ['TCTGGATAA', 'TGTGGATAA', 'TTATCCACA', 'TTATCCAGA'],
 ['TGATCAACA', 'TGTTGATCA'],
 ['TGTGAATAA', 'TTATTCACA']]